In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.models as model
from torch.optim.lr_scheduler import ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm

# CONFIG CLASS

In [ ]:
class CFG:
    DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED= 1234
    PROBLEM= 'regression'
    MODEL_NAME= 'efficientnet_b3'
    IMG_SIZE= 512
    IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
    IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB
    N_FOLDS= 5
    LEARNING_RATE= 1e-3
    WEIGHT_DECAY= 0
    T_MAX= 10
    T_0= 5
    ETA_MIN= 0
    SCHEDULER= 'CosineAnnealingLR'
    BATCH_SIZE= 16
    BATCH_SIZE_TEST= 4
    EPOCHS= 5

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic= True

    
set_seed(CFG.SEED)

In [ ]:
train= pd.read_csv('../input/pawpreds-eda-fe-folds-and-stacking-meta-data/train_folds.csv')
train.columns

In [ ]:
meta_features= ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur', 'width', 'height']

# DATASET CLASS

In [ ]:
class PawDataset:
    def __init__(self, df, augmentations= None):
        self.df= df
        self.image_paths= df['image_path'].values
        self.targets= df['Pawpularity'].values
        self.meta_df= df[meta_features].values           #numpy arrays
        if (CFG.PROBLEM == 'classification'):
            self.targets= df['Pawpularity'].values / 100
        self.augmentations= augmentations
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        target= self.targets[idx]
        meta_data= self.meta_df[idx, :]
        image= cv2.imread(self.image_paths[idx])  #BGR 
        image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #RGB
        if self.augmentations:
            image= self.augmentations(image= image)['image']
        return image, torch.tensor(meta_data, dtype= torch.float), torch.tensor(target, dtype= torch.float)

In [ ]:
augmentations= {
    'train': A.Compose([
        A.RandomResizedCrop(height= CFG.IMG_SIZE, width= CFG.IMG_SIZE, scale= (0.85, 1.0)),
        A.Flip(p= 0.7),
        A.Perspective(p= 0.7),
        A.Rotate(limit= 40, p= 0.5, border_mode= cv2.BORDER_CONSTANT),
        A.Normalize(mean= CFG.IMAGENET_MEAN, std= CFG.IMAGENET_STD),
        ToTensorV2()]),
    
    'valid': A.Compose([
        A.Resize(height= CFG.IMG_SIZE, width= CFG.IMG_SIZE),
        A.Normalize(mean= CFG.IMAGENET_MEAN, std= CFG.IMAGENET_STD),
        ToTensorV2()])
}

# MODEL

In [ ]:
class PawpularityModel(nn.Module):
    def __init__(self, model_name, pretrained= True):
        super(PawpularityModel, self).__init__()
        self.image_model= timm.create_model(model_name, pretrained= pretrained)
        self.input_dim= self.image_model.classifier.in_features
        self.image_model.classifier= nn.Identity() 
        
        self.fc1= nn.Linear(self.input_dim, self.input_dim//2)
        self.fc2= nn.Linear(self.input_dim//2, 1)
        
    def forward(self, image):
        emb= self.image_model(image)    #[N, 1280]
        output= F.relu(self.fc1(emb))
        output= torch.flatten(self.fc2(output))       #[N]
        return emb, output

# LOSS FUNCTION

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self, eps= 1e-8):
        super(RMSELoss, self).__init__()
        self.mse= nn.MSELoss()
        self.eps= eps
    
    def forward(self, output, target):
        loss= torch.sqrt(self.mse(output, target)+ self.eps)
        return loss

In [ ]:
def get_criterion():
    if CFG.PROBLEM == 'regression':
        return RMSELoss()
    
    if CFG.PROBLEM == 'classification':
        return nn.BCEWithLogitsLoss()

In [ ]:
def get_scheduler(optimizer):
    if CFG.SCHEDULER == 'ReduceLROnPlateau':
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    
    if CFG.SCHEDULER == 'CosineAnnealingLR':
        return optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= CFG.T_MAX, eta_min= CFG.ETA_MIN, verbose= True)
        
    if CFG.SCHEDULER== 'CosineAnnealingWarmRestarts':
        return optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= CFG.T_0, eta_min= CFG.ETA_MIN, verbose= True)
        

In [ ]:
def train_model(dataloader, model, criterion, scheduler):
    model.train()
    n_batches= len(dataloader)
    train_loss= 0
    
    for image, meta, target in dataloader:
        image= image.to(CFG.DEVICE)  #[N, C, H, W]
        meta= meta.to(CFG.DEVICE)
        target= target.to(CFG.DEVICE)
        
        _, output= model(image)
        loss= criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss
    
    train_loss_epoch= train_loss/n_batches
    return train_loss_epoch

In [ ]:
def val_model(dataloader, model, criterion):
    model.eval()
    n_batches= len(dataloader)
    val_loss= 0
    
    with torch.no_grad():
        for image, meta, target in dataloader:
            image= image.to(CFG.DEVICE)
            meta= meta.to(CFG.DEVICE)
            target= target.to(CFG.DEVICE)

            _, output= model(image)
            loss= criterion(output, target)
            
            val_loss += loss
    
    val_loss_epoch= val_loss/n_batches
    return val_loss_epoch

In [ ]:
def predict(dataloader, model):
    model.eval()
    preds= np.array([])
    
    with torch.no_grad():
        for image, meta, target in dataloader:
            image= image.to(CFG.DEVICE)
            meta= meta.to(CFG.DEVICE)
            target= target.to(CFG.DEVICE)

            _, output= model(image)
            
            if CFG.PROBLEM== 'classification':
                output= torch.sigmoid(output)*100
                
            output= output.cpu().numpy()
            preds= np.concatenate((preds, output))
            
    return preds

# TRAINING

In [ ]:
def get_data_loaders(train_idx, val_idx):
    train_data, val_data= train.iloc[train_idx], train.iloc[val_idx]

    train_dataset= PawDataset(train_data, augmentations= augmentations['train'])
    val_dataset= PawDataset(val_data, augmentations= augmentations['valid'])
    
    train_loader= data.DataLoader(train_dataset, batch_size= CFG.BATCH_SIZE)
    val_loader= data.DataLoader(val_dataset, batch_size= CFG.BATCH_SIZE)
    
    return train_loader, val_loader

In [ ]:
oof_preds= np.zeros((train.shape[0]))

for fold in range(CFG.N_FOLDS):
    train_idx= train[train['kfold']!= fold].index
    val_idx= train[train['kfold']== fold].index
    train_loader, val_loader= get_data_loaders(train_idx, val_idx)
    
    model= PawpularityModel(CFG.MODEL_NAME).to(CFG.DEVICE)
    criterion= get_criterion().to(CFG.DEVICE)
    optimizer= optim.Adam(model.parameters(), lr= CFG.LEARNING_RATE, weight_decay= CFG.WEIGHT_DECAY)
    scheduler= get_scheduler(optimizer)
    
    for t in range(CFG.EPOCHS):
        train_loss_epoch= train_model(train_loader, model, criterion, optimizer)
        val_loss_epoch= val_model(val_loader, model, criterion)
        print(f"FOLD: {fold}, EPOCH:{t}, train_loss:{train_loss_epoch}, val_loss:{val_loss_epoch}")
        
        scheduler.step()
    
    ofilename= "effnetb3_rmse%d.pth" % fold
    torch.save(model.state_dict(), ofilename)
    model.load_state_dict(torch.load('effnetb3_rmse%d.pth' % fold, map_location= CFG.DEVICE))
    oof_preds[val_idx]= predict(val_loader, model)

In [ ]:
oof_preds.shape

In [ ]:
oof_loss= mean_squared_error(train['Pawpularity'].to_numpy(), oof_preds, squared= False)
oof_loss